### Zillow Scraper Handles Requests
This document will only serve as a scraper that is necessary to retrieve the missing home prices, ourselves, minus the O'Hare Community given that much of the area is an airport. Other than that, feel free to utilize this scraper and reference `https://app.scrapeak.com`

~ Scraper will handle requests, manually input community/city of interest in format https://zillow.com/url/orlano-florida/etc. ~ Thank you.

In [1]:
# scrape the missing housing price indices
import pandas as pd
import numpy as np
import time
import requests
import plotly.express as px
import warnings

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

In [20]:
target = [
 'GRAND BOULEVARD',
 'DOUGLAS',
 'BELMONT CRAGIN',
 'AUSTIN',
 'NORTH LAWNDALE',
 'SOUTH LAWNDALE',
 'LOWER WEST SIDE',
 'NEAR SOUTH SIDE',
 'ARMOUR SQUARE',
 'NORWOOD PARK',
 'NEAR NORTH SIDE',
 'LOOP',
 'RIVERDALE',
 'NEW CITY',
 'GREATER GRAND CROSSING',
 'AUBURN GRESHAM',
 'OHARE']

In [5]:
api_key = "YOUR-API-KEY"

In [99]:
# get listings
url = 'KENSUKE_FUKUTOMI.COM/URL/HTML'
listing_response = get_listings(api_key, url)
listing_response.json().keys()
listing_response.json()["is_success"]

True

In [100]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 255


In [101]:
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 248
Number of columns: 79


,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,beds,baths,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,brokerName,timeOnZillow,latLong.latitude,latLong.longitude,variableData.type,variableData.text,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.dateSold,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,area,hdpData.homeInfo.livingArea,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.priceReduction,hdpData.homeInfo.unit,hdpData.homeInfo.contingentListingType,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,lotAreaString,streetViewURL,streetViewMetadataURL
0,2070918984,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/3c7db6c8b2c...,True,/homedetails/744-W-77th-St-Chicago-IL-60620/20...,SOLD,Closed,"$440,000",$440K,"744 W 77th St, Chicago, IL 60620",8.0,4.0,False,False,RecentlySold,RecentChange,False,False,False,False,True,False,False,,False,False,Fulton Grace Realty,275839000,41.754414,-87.643480,RECENTLY_SOLD,Sold 04/23/2024,2070918984,744 W 77th St,60620,Chicago,IL,41.754414,-87.643480,440000.0,1713855600000,4.0,8.0,MULTI_FAMILY,RECENTLY_SOLD,-1,False,False,420200.0,2605,False,False,RECENTLY_SOLD,440000.0,275839000,True,False,False,USD,USA,1742.400,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4008339,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/5c9686e1722...,True,/homedetails/7243-S-May-St-Chicago-IL-60621/40...,SOLD,Closed,"$232,500",$233K,"7243 S May St, Chicago, IL 60621",3.0,2.0,False,False,RecentlySold,RecentChange,False,False,False,False,True,False,False,,False,False,PROSALES REALTY,794239000,41.761970,-87.652470,RECENTLY_SOLD,Sold 04/17/2024,4008339,7243 S May St,60621,Chicago,IL,41.761970,-87.652470,232500.0,1713337200000,2.0,3.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,232700.0,2122,False,False,RECENTLY_SOLD,232500.0,794239000,True,False,False,USD,USA,3123.252,sqft,False,1908.0,1908.0,36580.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4120602,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/02b00f5b260...,True,/homedetails/8957-S-Justine-St-Chicago-IL-6062...,SOLD,Closed,"$120,000",$120K,"8957 S Justine St, Chicago, IL 60620",4.0,2.0,False,False,RecentlySold,RecentChange,False,False,False,False,True,False,False,,False,False,Pearson Realty Group,794239000,41.730488,-87.661350,RECENTLY_SOLD,Sold 04/17/2024,4120602,8957 S Justine St,60620,Chicago,IL,41.730488,-87.661350,120000.0,1713337200000,2.0,4.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,120000.0,2047,False,False,RECENTLY_SOLD,120000.0,794239000,True,False,False,USD,USA,3567.000,sqft,False,1112.0,1112.0,121760.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4014333,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/27ad01f53ea...,True,/homedetails/8616-S-Hermitage-Ave-Chicago-IL-6...,SOLD,Closed,

In [18]:
prices = [] # DO NOT RESET

In [102]:
# this portion will obtain the median pricing for the community
prices.append(float(df_listings['hdpData.homeInfo.price'].median()))

In [104]:
# for the Chicago, OHARE Community ONLY
prices.append(0.0)

In [105]:
prices

[275000.0,
 355000.0,
 415000.0,
 369000.0,
 244000.0,
 255000.0,
 423400.0,
 466500.0,
 455500.0,
 420000.0,
 605000.0,
 497450.0,
 52500.0,
 52500.0,
 153500.0,
 230000.0,
 0.0]

In [107]:
hash = {"Community Name" : target, "Price" : prices}
df = pd.DataFrame(hash)

In [110]:
df.to_csv('missing_home_prices.csv', index=False)